ディレクトリ・モデルの指定

In [1]:
# リポジトリを置くところ
home_dir = '/content/drive/My\ Drive'
# データを入れたディレクトリの名前
data_dir='pumpkin_640_480'
# ベースモデル: yolox_nano, yolox_tiny, yolox_s, yolox_m, yolox_l, yolox_x
model_type = "yolox_s"

(初回のみ) インストール

In [ ]:
%cd {home_dir}
!git clone https://github.com/husty530/Yamashita-yolox.git
%cd {home_dir}/Yamashita-yolox
!pip3 install -U pip && pip3 install -r requirements.txt
!pip3 install -v -e .
!pip3 install cython; pip3 install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

学習条件の設定

In [ ]:
###### config ######

width = 640
height = 480
num_classes = 1
max_epoch = 50
data_num_workers = 2
interval = 5

####################

%cd {home_dir}/Yamashita-yolox
%cp yolox/{model_type}.py {model_type}.py
!python3 config.py {model_type} {home_dir} {data_dir} {num_classes} {max_epoch} {data_num_workers} {interval} {width} {height}
url = f'https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/{model_type}.pth'
!wget {url} -O {model_type}.pth

(新規・データ変更時)train-val の切り分け -- YOLOフォーマット(space区切り.txtファイル)のみ対応

In [ ]:
%cd {home_dir}/Yamashita-yolox/workspace/{data_dir}
!rm -r train2017
!mkdir train2017
!rm -r val2017
!mkdir val2017
!rm -r annotations
!mkdir annotations

percentage_valid = 20
!python3 ../../split.py {percentage_valid} {width} {height}

学習開始

In [ ]:
%cd {home_dir}/Yamashita-yolox
!pip3 install loguru thop
!cp yolox/train.py train.py

batch_size = 16
!python3 train.py -f {model_type}.py -d 1 -b {batch_size} -o -c {model_type}.pth
!rm train.py

デモ (workspace/{フォルダ}/{モデル名}/vis_res/ 以下に保存されます)

In [ ]:
%cd {home_dir}/Yamashita-yolox
!pip3 install loguru thop
!cp yolox/demo.py demo.py

!python3 demo.py image -f {model_type}.py --device gpu\
    -c workspace/{data_dir}/{model_type}/best_ckpt.pth\
    --path workspace/{data_dir}/val2017/004.png\
    --names workspace/{data_dir}/.names\
    --conf 0.5 --nms 0.65 --save_result
!rm demo.py

評価

In [ ]:
%cd {home_dir}/Yamashita-yolox
!pip3 install loguru thop
!cp yolox/eval.py eval.py

batch_size = 16
!python3 eval.py -f {model_type}.py -d 1 -b {batch_size}\
    -c workspace/{data_dir}/{model_type}/best_ckpt.pth\
    --names workspace/{data_dir}/.names\
    --conf 0.5 --nms 0.65
!rm eval.py

ONNXエクスポート

In [ ]:
%cd {home_dir}/Yamashita-yolox
!pip3 install loguru thop onnx onnx-simplifier
!cp yolox/export_onnx.py export_onnx.py
!python3 export_onnx.py -f {model_type}.py -c workspace/{data_dir}/{model_type}/best_ckpt.pth --output-name workspace/{data_dir}/{model_type}.onnx
!rm export_onnx.py